## FireSQL Programming Interface
We provided a simple firebase SQL interface class that can be easily applied a SQL statement to fetch from Firebase collections.

In [1]:
from firesql.firebase import FirebaseClient

client = FirebaseClient()
client.connect(credentials_json='../credentials/credentials.json')


In [2]:
from firesql.sql import FireSQL
from firesql.sql.sql_fire_client import FireSQLClient
import pandas as pd

fireSQL = FireSQL()
sqlClient = FireSQLClient(client)

In [3]:
company = 'bennycorp'

sql1 = """
  SELECT docid, email, state
    FROM
      Companies/{}/Users
    WHERE
      state IN ('ACTIVE')
""".format(company)

sql2 = """
  SELECT *
    FROM
      Companies/{}/Users
""".format(company)

sql3 = """
  SELECT u.docid, u.email, u.state, b.date, b.locationId, b.state
    FROM
      Companies/{}/Users as u JOIN Companies/{}/Bookings as b
      ON u.email = b.email
    WHERE 
        u.state IN ('ACTIVE') and
        b.state IN ('CHECKED_IN', 'CHECKED_OUT') and
        b.date >= '2022-03-18T04:00:00'
""".format(company, company)

sql4 = """
  SELECT docid, email, state
    FROM
      Companies/{}/Users
    WHERE
      state IN ('ACTIVE') and
      email NOT LIKE '%twotwo%'
""".format(company)

selectDocs = fireSQL.sql(sqlClient, sql3)
df = pd.DataFrame(selectDocs)
df

,date,locationId,state,docid,email,u_state
0,2022-03-18 04:00:00+00:00,BrACWIg7Ar3xWgOU3Fdv,CHECKED_IN,1utcUa9fdheOlrMe9GOCjrJ3wjh1,btscheung+bennycorp@gmail.com,ACTIVE
1,2022-03-18 04:00:00+00:00,BrACWIg7Ar3xWgOU3Fdv,CHECKED_IN,1utcUa9fdheOlrMe9GOCjrJ3wjh1,btscheung+bennycorp@gmail.com,ACTIVE
2,2022-03-18 04:00:00+00:00,BrACWIg7Ar3xWgOU3Fdv,CHECKED_IN,1utcUa9fdheOlrMe9GOCjrJ3wjh1,btscheung+bennycorp@gmail.com,ACTIVE
3,2022-03-31 04:00:00+00:00,BrACWIg7Ar3xWgOU3Fdv,CHECKED_IN,xQDDc7SXpqXxRAxIQWb1xPqdGYw2,btscheung+hill6@gmail.com,ACTIVE
4,2022-04-13 04:00:00+00:00,BrACWIg7Ar3xWgOU3Fdv,CHECKED_IN,xQDDc7SXpqXxRAxIQWb1xPqdGYw2,btscheung+hill6@gmail.com,ACTIVE


In [4]:
company = 'bennycorp'

insert1 = """
  INSERT INTO Companies/{}/Users
    (email, state)
    VALUES ('btscheung+test1@gmail.com', 'ACTIVE')
""".format(company)

docs = fireSQL.sql(sqlClient, insert1)
df = pd.DataFrame(docs)
df

,email,state,docid
0,btscheung+test1@gmail.com,ACTIVE,3X2x0r0Vx3yHmuJZVrvF


In [5]:
company = 'bennycorp'

update1 = """
  UPDATE Companies/{}/Users
  SET state = 'INACTIVE'
  WHERE state = 'ACTIVE' and email = 'btscheung+test1@gmail.com'
""".format(company)

docs = fireSQL.sql(sqlClient, update1)
df = pd.DataFrame(docs)
df

,docid,events,state,email
0,3X2x0r0Vx3yHmuJZVrvF,"[{'event': 'CREATION', 'date': 2022-04-23 01:3...",INACTIVE,btscheung+test1@gmail.com


In [6]:
company = 'bennycorp'

delete1 = """
DELETE
  FROM Companies/{}/Users
  WHERE 
    email = 'btscheung+test1@gmail.com'

""".format(company)

docs = fireSQL.sql(sqlClient, delete1)
df = pd.DataFrame(docs)
df

,docid,state,email,events
0,3X2x0r0Vx3yHmuJZVrvF,INACTIVE,btscheung+test1@gmail.com,"[{'date': 2022-04-23 01:37:35.242000+00:00, 'e..."
